
<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<font color=0F5298 size=7>
Digital Image Processing <br>
<font color=2565AE size=5>
Department Of Mathematical Sciences <br>
Spring 2025<br>
<font color=3C99D size=5>
Assignment 3 <br>
</div>


---



### Full Name :
### Student Number :
___

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Step 1: Divide image into 8x8 blocks and apply DCT
## (use realimage.jpg)

In [ ]:
def apply_dct(image):
    dct_blocks = None
    # YOUR CODE
    return dct_blocks

## Step 2: Quantization


In [ ]:
# Quantization matrix

Z = np.array([[16,11,10,16,24,40,51,61],
              [12,12,14,19,26,58,60,55],
              [14,13,16,24,40,57,69,56],
              [14,17,22,29,51,87,80,62],
              [18,22,37,56,68,109,103,77],
              [24,35,55,64,81,104,113,92],
              [49,64,78,87,103,121,120,101],
              [72,92,95,98,112,100,130,99]])

In [ ]:
def quantize(dct_blocks):
    quantized_blocks = None
    # YOUR CODE
    return quantized_blocks

### Matrix to vector
Now you need to convert the matrix to an array like this:

`[-17, 0, -19, ..., 'EOB']`

In [ ]:
def mat_2_vec(jpeg_matrix):
    jpeg_vector = None
    # YOUR CODE
    return jpeg_vector

## Step 3: Apply Inverse DCT for decompression

In [ ]:
def apply_idct(quantized_blocks):
    decompressed_image = None
    # YOUR CODE
    return decompressed_image.astype(np.uint8)